In [21]:
from torchvision import datasets, transforms
import torch
import torch.nn.functional as F


# Hyperparams
batch_size = 50
loss_func = F.cross_entropy
epochs = 50

# GPU/CPU
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
use_cuda = torch.cuda.is_available()

# Datasets
train_kwargs = {'batch_size': batch_size, 'shuffle': True}
val_kwargs = {'batch_size': batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True}
    train_kwargs.update(cuda_kwargs)
    val_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
val_loader = torch.utils.data.DataLoader(dataset2, **val_kwargs)

In [28]:
import torch.nn as nn

# model
    
model = nn.Sequential(
    nn.Conv2d(1, 32, 5, 1, padding=(2, 2)),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 5, 1, padding=(2, 2)),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(7 * 7 * 64, 1024),
    nn.ReLU(),
    nn.Linear(1024, 10)
)

model.to(dev)

Sequential(
  (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=3136, out_features=1024, bias=True)
  (8): ReLU()
  (9): Linear(in_features=1024, out_features=10, bias=True)
)

In [29]:
# optimizer
import torch.optim as optim

learning_rate = 1e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [30]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

# Training
best_val_acc = 0

for epoch in range(epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(dev), target.to(dev)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        model.eval()
        val_acc = 0
        for batch_idx, (data, target) in enumerate(val_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_acc += accuracy(output, target)
    
        val_acc = val_acc / len(val_loader)

        print(f'Val Loss acc {epoch} epoch(s): {val_acc}')

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model, 'cnn_mnist')
        

KeyboardInterrupt: 